# Examen 2

In [94]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats

#Visualización

import cufflinks as cf
cf.go_offline()

# Predicción continua

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lars, Lasso, Ridge, ElasticNet, BayesianRidge, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#Clasificadores

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

In [2]:
datat = pd.read_csv('./data/train_e1.csv')

In [3]:
datat

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,filename_hr,NOT_NORM
0,16286,10874.0,68.0,0,NaN,NaN,0.0,0.0,CS100 3,1996-07-04 14:12:49,records500/16000/16286_hr,1
1,2647,20573.0,34.0,1,NaN,57.0,0.0,0.0,CS-12 E,1989-04-09 08:52:33,records500/02000/02647_hr,0
2,11732,12198.0,53.0,0,NaN,81.0,0.0,0.0,CS-12 E,1994-01-26 13:21:53,records500/11000/11732_hr,0
3,19751,20578.0,59.0,0,NaN,NaN,0.0,0.0,CS100 3,1998-10-08 15:19:32,records500/19000/19751_hr,1
4,7898,5104.0,69.0,1,160.0,59.0,6.0,1.0,AT-6 C 5.8,1992-03-22 19:11:05,records500/07000/07898_hr,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15280,15377,16237.0,47.0,0,NaN,71.0,0.0,0.0,CS-12 E,1995-11-25 10:18:13,records500/15000/15377_hr,0
15281,5475,17361.0,47.0,0,NaN,NaN,0.0,0.0,CS100 3,1990-12-06 09:29:17,records500/05000/05475_hr,0
15282,3790,13456.0,75.0,1,NaN,NaN,NaN,3.0,AT-6 C,1990-02-20 14:48:42,records500/03000/03790_hr,1
15283,21643,16888.0,72.0,1,NaN,NaN,0.0,0.0,CS100 3,2000-12-11 11:51:56,records500/21000/21643_hr,1


In [90]:
datat222 = pd.read_csv('./data/train_e1.csv')

In [4]:
def ecg(r):

    #Lectura de datos

    r2 = './data/'+r
    s, f = wfdb.rdsamp(r2)
    s = pd.DataFrame(s, columns=f['sig_name'])
    rpeaks = nk.ecg_findpeaks(s['II'], sampling_rate=500)

    #Detección de picos

    if len(rpeaks['ECG_R_Peaks']) < 5:
        print(f"El archivo {r} contiene menos de 5 picos R, omitiendo.")
        return pd.DataFrame({
            'record': r,
            'BPM': [np.nan], 'HRV': [np.nan], 'QT_int': [np.nan],
            'R_amplitude': [np.nan], 'QRS_width': [np.nan], 'PR_int': [np.nan]
        })

    waves = nk.ecg_delineate(s['II'], rpeaks, sampling_rate=500, method='dwt')
    rpeaks = np.array(rpeaks['ECG_R_Peaks'])

    #Variables
    #1. BPM

    bpm = nk.ecg_rate(rpeaks, sampling_rate=500, desired_length=len(s)).mean()

    #2. HRV

    hrv = nk.hrv_time(rpeaks, sampling_rate=500)
    hrv = hrv['HRV_SDNN']/1000

    #3. Intervalo QT
    wavesdict = waves[1]
    qt_int = (np.array(wavesdict['ECG_T_Offsets']) - np.array(wavesdict['ECG_Q_Peaks']))
    qt_int = qt_int[~pd.isnull(qt_int)] / 500
    qt_int = qt_int.mean()

    #4. Amplitud de la onda R

    r_amp = s['II'][rpeaks].max() - s['II'][rpeaks].min()

    #5. Ancho de QRS

    qrs = (np.array(wavesdict['ECG_R_Offsets']) - np.array(wavesdict['ECG_R_Onsets']))
    qrs = qrs[~pd.isnull(qrs)] / 500
    qrs = qrs.mean()

    #6. Intervalo PR

    pr = (np.array(wavesdict['ECG_R_Onsets']) - np.array(wavesdict['ECG_P_Onsets'])) / 500
    pr = pr[~pd.isnull(pr)].mean()

    d = pd.DataFrame({'record': r,'BPM': bpm, 'HRV': hrv, 'QT_int': qt_int, 'R_amplitude': r_amp, 'QRS_width': qrs, 'PR_int': pr}, index=[0]) 
    
    return d

In [5]:
resultados = pd.DataFrame()

for r in datat['filename_hr']:
    resultados = pd.concat([resultados, ecg(r)], axis=0)

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfi

El archivo records500/12000/12698_hr contiene menos de 5 picos R, omitiendo.


/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



El archivo records500/16000/16118_hr contiene menos de 5 picos R, omitiendo.


/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfi

...

El archivo records500/09000/09317_hr contiene menos de 5 picos R, omitiendo.


/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:52: RuntimeWarning:

Mean of empty slice.

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8l

El archivo records500/04000/04368_hr contiene menos de 5 picos R, omitiendo.
El archivo records500/15000/15973_hr contiene menos de 5 picos R, omitiendo.
El archivo records500/20000/20392_hr contiene menos de 5 picos R, omitiendo.


/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



El archivo records500/03000/03355_hr contiene menos de 5 picos R, omitiendo.
El archivo records500/07000/07514_hr contiene menos de 5 picos R, omitiendo.
El archivo records500/02000/02201_hr contiene menos de 5 picos R, omitiendo.
El archivo records500/14000/14359_hr contiene menos de 5 picos R, omitiendo.


In [6]:
resultados

,record,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int
0,records500/16000/16286_hr,76.564181,0.009693,0.191455,0.112,0.073455,0.146545
0,records500/02000/02647_hr,48.891056,0.054899,0.440000,0.285,0.098857,0.134571
0,records500/11000/11732_hr,60.514093,0.039332,0.416222,0.430,0.130889,0.070667
0,records500/19000/19751_hr,68.134493,0.006613,0.339200,0.155,0.096800,0.140800
0,records500/07000/07898_hr,56.967957,0.012556,0.426250,0.048,0.357143,0.065750
...,...,...,...,...,...,...,...
0,records500/15000/15377_hr,59.087159,0.031838,0.402750,0.095,0.166000,0.079750
0,records500/05000/05475_hr,76.600072,0.168882,0.349000,1.244,0.133500,0.130250
0,records500/03000/03790_hr,70.944370,0.015513,0.372545,0.346,0.115455,0.109091
0,records500/21000/21643_hr,62.791343,0.040981,0.498889,0.279,0.163111,0.114444


In [7]:
datat = datat.merge(resultados, left_on='filename_hr', right_on='record', how = 'left')

In [8]:
datat = datat.drop(columns=['record'])

In [10]:
datat

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,filename_hr,NOT_NORM,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int
0,16286,10874.0,68.0,0,NaN,NaN,0.0,0.0,CS100 3,1996-07-04 14:12:49,records500/16000/16286_hr,1,76.564181,0.009693,0.191455,0.112,0.073455,0.146545
1,2647,20573.0,34.0,1,NaN,57.0,0.0,0.0,CS-12 E,1989-04-09 08:52:33,records500/02000/02647_hr,0,48.891056,0.054899,0.440000,0.285,0.098857,0.134571
2,11732,12198.0,53.0,0,NaN,81.0,0.0,0.0,CS-12 E,1994-01-26 13:21:53,records500/11000/11732_hr,0,60.514093,0.039332,0.416222,0.430,0.130889,0.070667
3,19751,20578.0,59.0,0,NaN,NaN,0.0,0.0,CS100 3,1998-10-08 15:19:32,records500/19000/19751_hr,1,68.134493,0.006613,0.339200,0.155,0.096800,0.140800
4,7898,5104.0,69.0,1,160.0,59.0,6.0,1.0,AT-6 C 5.8,1992-03-22 19:11:05,records500/07000/07898_hr,0,56.967957,0.012556,0.426250,0.048,0.357143,0.065750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,15377,16237.0,47.0,0,NaN,71.0,0.0,0.0,CS-12 E,1995-11-25 10:18:13,records500/15000/15377_hr,0,59.087159,0.031838,0.402750,0.095,0.166000,0.079750
15281,5475,17361.0,47.0,0,NaN,NaN,0.0,0.0,CS100 3,1990-12-06 09:29:17,records500/05000/05475_hr,0,76.600072,0.168882,0.349000,1.244,0.133500,0.130250
15282,3790,13456.0,75.0,1,NaN,NaN,NaN,3.0,AT-6 C,1990-02-20 14:48:42,records500/03000/03790_hr,1,70.944370,0.015513,0.372545,0.346,0.115455,0.109091
15283,21643,16888.0,72.0,1,NaN,NaN,0.0,0.0,CS100 3,2000-12-11 11:51:56,records500/21000/21643_hr,1,62.791343,0.040981,0.498889,0.279,0.163111,0.114444


## Nulls

In [11]:
datat.isnull().sum() / len(datat)

ecg_id            0.000000
patient_id        0.000000
age               0.003991
sex               0.000000
height            0.682761
weight            0.569055
nurse             0.067386
site              0.000981
device            0.000000
recording_date    0.000000
filename_hr       0.000000
NOT_NORM          0.000000
BPM               0.001112
HRV               0.001112
QT_int            0.004907
R_amplitude       0.001112
QRS_width         0.005692
PR_int            0.001963
dtype: float64

In [12]:
#Imputa la moda en la edad según el sexo

datat.loc[(datat['age'].isnull()) & (datat['sex'] == 0), 'age'] = datat[datat['sex'] == 0]['age'].mode()[0]

In [13]:
datat.loc[(datat['age'].isnull()) & (datat['sex'] == 1), 'age'] = datat[datat['sex'] == 1]['age'].mode()[0]

In [14]:
missings = ['BPM', 'HRV', 'QT_int', 'R_amplitude', 'QRS_width', 'PR_int']

datat[missings] = datat[missings].fillna(datat[missings].median())

In [15]:
datat['nurse'] = datat['nurse'].fillna(datat['nurse'].mode()[0])

datat['site'] = datat['site'].fillna(datat['site'].mode()[0])

### Regresor para peso - altura

In [16]:
wh = datat[(datat['weight'].notnull()) & (datat['height'].notnull())][['patient_id', 'age', 'sex', 'weight', 'height']].copy()


In [17]:
whnull = datat[(datat['weight'].isnull()) | (datat['height'].isnull())][['patient_id', 'age', 'sex', 'weight', 'height']].copy()

In [18]:
wh

,patient_id,age,sex,weight,height
4,5104.0,69.0,1,59.0,160.0
5,7033.0,69.0,0,92.0,173.0
7,5344.0,83.0,0,92.0,193.0
10,734.0,70.0,1,50.0,164.0
13,1320.0,87.0,0,57.0,164.0
...,...,...,...,...,...
15254,3603.0,39.0,1,50.0,155.0
15262,5785.0,71.0,0,66.0,173.0
15264,1025.0,49.0,0,82.0,168.0
15265,4558.0,78.0,1,75.0,152.0


In [19]:
#Modelo 1: target = weight

X = wh.drop(columns=['weight', 'patient_id'])

y = wh['weight']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
models = {'LR': LinearRegression(), 'Lars': Lars(), 'Lasso': Lasso(), 'Ridge': Ridge(), 'ElasticNet': ElasticNet(), 'BayesianRidge': BayesianRidge(), 'SGDRegressor': SGDRegressor()}
scalers = {'StandardScaler': StandardScaler(), 'MinMaxScaler': MinMaxScaler(), 'RobustScaler': RobustScaler()}

results = pd.DataFrame()

for model_name, model in models.items():
    for scaler_name, scaler in scalers.items():
        pipe = Pipeline(steps=[('scaler', scaler), ('model', model)])
        pipe.fit(X_train, y_train)
        #Cross validation
        cv_score = cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2')
        scoretrain = pipe.score(X_train, y_train)   
        scoretest = pipe.score(X_test, y_test)
        new_row = pd.DataFrame({'model': [model_name], 'scaler': [scaler_name], 'train_score': [scoretrain], 'test_score': [scoretest], 'cv_score': [cv_score.mean()], 'cv_score_std': [cv_score.std()]})
        results = pd.concat([results, new_row], ignore_index=True)

In [21]:
results.sort_values(by='test_score', ascending=False)

,model,scaler,train_score,test_score,cv_score,cv_score_std
18,SGDRegressor,StandardScaler,0.332037,0.357132,0.329127,0.034720
4,Lars,MinMaxScaler,0.332212,0.356656,0.329043,0.036130
5,Lars,RobustScaler,0.332212,0.356656,0.329043,0.036130
1,LR,MinMaxScaler,0.332212,0.356656,0.329043,0.036130
0,LR,StandardScaler,0.332212,0.356656,0.329043,0.036130
2,LR,RobustScaler,0.332212,0.356656,0.329043,0.036130
3,Lars,StandardScaler,0.332212,0.356656,0.329043,0.036130
9,Ridge,StandardScaler,0.332212,0.356644,0.329044,0.036118
11,Ridge,RobustScaler,0.332212,0.356634,0.329047,0.036103
17,BayesianRidge,RobustScaler,0.332211,0.356566,0.329051,0.036026


In [22]:
#SGDRegressor StandardScaler GridSearchCV

pipe = Pipeline(steps=[('scaler', StandardScaler()), ('model', SGDRegressor())])

In [23]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', SGDRegressor())])

In [24]:
#Score en train

pipe.score(X_train, y_train)

0.33169822396153403

In [25]:
#Score en test

pipe.score(X_test, y_test)

0.3536843103813192

In [116]:
#Modelo 2: target = height

X = wh.drop(columns=['height', 'patient_id'])

y = wh['height']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [27]:
results2 = pd.DataFrame()

for model_name, model in models.items():
    for scaler_name, scaler in scalers.items():
        pipe = Pipeline(steps=[('scaler', scaler), ('model', model)])
        pipe.fit(X_train, y_train)
        #Cross validation
        cv_score = cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2')
        scoretrain = pipe.score(X_train, y_train)   
        scoretest = pipe.score(X_test, y_test)
        new_row = pd.DataFrame({'model': [model_name], 'scaler': [scaler_name], 'train_score': [scoretrain], 'test_score': [scoretest], 'cv_score': [cv_score.mean()], 'cv_score_std': [cv_score.std()]})
        results2 = pd.concat([results2, new_row], ignore_index=True)

In [28]:
results2.sort_values(by='test_score', ascending=False)

,model,scaler,train_score,test_score,cv_score,cv_score_std
18,SGDRegressor,StandardScaler,0.470607,0.523659,0.466505,0.039381
10,Ridge,MinMaxScaler,0.470571,0.523165,0.465275,0.040857
16,BayesianRidge,MinMaxScaler,0.470869,0.523120,0.465354,0.041221
4,Lars,MinMaxScaler,0.470871,0.523089,0.465350,0.041246
5,Lars,RobustScaler,0.470871,0.523089,0.465350,0.041246
1,LR,MinMaxScaler,0.470871,0.523089,0.465350,0.041246
0,LR,StandardScaler,0.470871,0.523089,0.465350,0.041246
3,Lars,StandardScaler,0.470871,0.523089,0.465350,0.041246
2,LR,RobustScaler,0.470871,0.523089,0.465350,0.041246
9,Ridge,StandardScaler,0.470871,0.523086,0.465352,0.041231


In [29]:
#Grafica el peso y la altura de wh con cufflinks

wh.iplot(kind='scatter', x='weight', y='height', mode='markers', xTitle='Peso', yTitle='Altura')

In [30]:
#Histograma peso wh

wh['weight'].iplot(kind='hist', bins=100, xTitle='Peso', yTitle='Frecuencia')

In [31]:
#Histograma altura wh

wh['height'].iplot(kind='hist', bins=100, xTitle='Altura', yTitle='Frecuencia')

In [32]:
#Histograma edad

wh['age'].iplot(kind='hist', bins=100, xTitle='Edad', yTitle='Frecuencia')

In [33]:
#Pruebas de ajuste a una distribución normal

#Peso Kolmogorov - Smirnov

weightdata = datat['weight'].dropna()

ksw = stats.kstest(weightdata, 'norm', args=(weightdata.mean(), weightdata.std()))

In [34]:
ksw

KstestResult(statistic=0.06436893390311604, pvalue=3.5933193680946186e-24, statistic_location=70.0, statistic_sign=1)

In [35]:
#Altura Kolmogorov - Smirnov

heightdata = datat['height'].dropna()

ksh = stats.kstest(heightdata, 'norm', args=(heightdata.mean(), heightdata.std()))

In [36]:
ksh

KstestResult(statistic=0.05110310030043541, pvalue=1.9104582527793808e-11, statistic_location=170.0, statistic_sign=1)

In [37]:
#Kolmogorov - Smirnov Gamma y Lognorm

kswg = stats.kstest(weightdata, 'gamma', args=(weightdata.mean(), weightdata.std()))
kswl = stats.kstest(weightdata, 'lognorm', args=(weightdata.mean(), weightdata.std()))

kshg = stats.kstest(heightdata, 'gamma', args=(heightdata.mean(), heightdata.std()))
kshl = stats.kstest(heightdata, 'lognorm', args=(heightdata.mean(), heightdata.std()))

kswg, kswl, kshg, kshl

(KstestResult(statistic=0.5856937737258131, pvalue=0.0, statistic_location=76.0, statistic_sign=1),
 KstestResult(statistic=0.5152815348184981, pvalue=0.0, statistic_location=35.0, statistic_sign=-1),
 KstestResult(statistic=0.3846927819032371, pvalue=0.0, statistic_location=170.0, statistic_sign=1),
 KstestResult(statistic=0.5095124231084529, pvalue=0.0, statistic_location=80.0, statistic_sign=-1))

In [38]:
prueba = datat[['height', 'weight']].copy()

In [39]:
from sklearn.impute import KNNImputer

In [40]:
prueba

,height,weight
0,NaN,NaN
1,NaN,57.0
2,NaN,81.0
3,NaN,NaN
4,160.0,59.0
...,...,...
15280,NaN,71.0
15281,NaN,NaN
15282,NaN,NaN
15283,NaN,NaN


In [41]:
knni = KNNImputer(n_neighbors=5)

prueba = knni.fit_transform(prueba)

In [42]:
prueba = pd.DataFrame(prueba, columns=['height', 'weight'])

In [43]:
prueba

,height,weight
0,166.798103,70.842872
1,157.600000,57.000000
2,174.400000,81.000000
3,166.798103,70.842872
4,160.000000,59.000000
...,...,...
15280,169.200000,71.000000
15281,166.798103,70.842872
15282,166.798103,70.842872
15283,166.798103,70.842872


In [44]:
#Histograma peso

prueba['weight'].iplot(kind='hist', bins=25, xTitle='Peso', yTitle='Frecuencia')

In [45]:
#hsiograma altura

prueba['height'].iplot(kind='hist', bins=25, xTitle='Altura', yTitle='Frecuencia')

In [46]:
#Prueba 2: con el modelo predictivo de altua SGDRegressor RobustScaler y = height

prueba2 = datat[['sex', 'age', 'weight', 'height']].copy()

In [117]:
pipe = Pipeline(steps=[('scaler', RobustScaler()), ('model', SGDRegressor())])

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()), ('model', SGDRegressor())])

In [118]:
#Score en train

pipe.score(X_train, y_train)

0.4708211988005565

In [119]:
#Score en test

pipe.score(X_test, y_test)

0.5234698812416473

In [50]:
prueba2['weight'] = prueba2['weight'].fillna(prueba2['weight'].median())

In [51]:
prueba2.columns = ['age', 'sex', 'weight', 'height']

In [52]:
prueba2['height'] = np.where(prueba2['height'].isnull(), pipe.predict(prueba2.drop(columns=['height'])), prueba2['height'])

In [53]:
#Histograma peso

prueba2['weight'].iplot(kind='hist', bins=25, xTitle='Peso', yTitle='Frecuencia')

In [54]:
#Histograma altura

prueba2['height'].iplot(kind='hist', bins=25, xTitle='Altura', yTitle='Frecuencia')

In [55]:
dfcor = datat[(datat['weight'].notnull()) & (datat['height'].notnull())].copy()

In [56]:
#Correlación entre las variables

dfcor = dfcor.drop(columns=['filename_hr', 'patient_id', 'ecg_id', 'recording_date', 'device', 'filename_hr'], axis=1)

In [57]:
corr = dfcor.corr()

In [58]:
#Heatmap

corr.iplot(kind='heatmap', colorscale='reds', title='Correlación entre variables')

In [59]:
dataset = datat.copy()

In [60]:
#Prueba 3


prueba3 = datat[['patient_id','age', 'sex', 'weight', 'height']].copy()

In [61]:
prueba3

,patient_id,age,sex,weight,height
0,10874.0,68.0,0,NaN,NaN
1,20573.0,34.0,1,57.0,NaN
2,12198.0,53.0,0,81.0,NaN
3,20578.0,59.0,0,NaN,NaN
4,5104.0,69.0,1,59.0,160.0
...,...,...,...,...,...
15280,16237.0,47.0,0,71.0,NaN
15281,17361.0,47.0,0,NaN,NaN
15282,13456.0,75.0,1,NaN,NaN
15283,16888.0,72.0,1,NaN,NaN


In [62]:
for a in prueba3['age']:
    if a < 18:
        prueba3['age'] = prueba3['age'].replace(a, '<18')
    elif a >= 18 and a < 30:
        prueba3['age'] = prueba3['age'].replace(a, '18-30')
    elif a >= 30 and a < 40:
        prueba3['age'] = prueba3['age'].replace(a, '30-40')
    elif a >= 40 and a < 50:
        prueba3['age'] = prueba3['age'].replace(a, '40-50')
    elif a >= 50 and a < 60:
        prueba3['age'] = prueba3['age'].replace(a, '50-60')
    elif a >= 60 and a < 70:
        prueba3['age'] = prueba3['age'].replace(a, '60-70')
    else:
        prueba3['age'] = prueba3['age'].replace(a, '>70')

In [63]:
prueba3

,patient_id,age,sex,weight,height
0,10874.0,60-70,0,NaN,NaN
1,20573.0,30-40,1,57.0,NaN
2,12198.0,50-60,0,81.0,NaN
3,20578.0,50-60,0,NaN,NaN
4,5104.0,60-70,1,59.0,160.0
...,...,...,...,...,...
15280,16237.0,40-50,0,71.0,NaN
15281,17361.0,40-50,0,NaN,NaN
15282,13456.0,>70,1,NaN,NaN
15283,16888.0,>70,1,NaN,NaN


In [64]:
dicpeso = {'<18': prueba3[prueba3['age'] == '<18']['weight'].dropna(), '18-30': prueba3[prueba3['age'] == '18-30']['weight'].dropna(), '30-40': prueba3[prueba3['age'] == '30-40']['weight'].dropna(), '40-50': prueba3[prueba3['age'] == '40-50']['weight'].dropna(), '50-60': prueba3[prueba3['age'] == '50-60']['weight'].dropna(), '60-70': prueba3[prueba3['age'] == '60-70']['weight'].dropna(), '>70': prueba3[prueba3['age'] == '>70']['weight'].dropna()}

In [65]:
#Donde haya nulos en weight, imputa una muestra aleatoria del grupo edad correspondiente en dicpeso

for a in dicpeso.keys():
    # Encuentra el número de valores nulos para la edad actual
    num_missing = prueba3[(prueba3['age'] == a) & (prueba3['weight'].isnull())].shape[0]
    
    if num_missing > 0:  # Solo proceder si hay valores nulos
        # Imputar valores aleatorios desde dicpeso[a]
        prueba3.loc[(prueba3['age'] == a) & (prueba3['weight'].isnull()), 'weight'] = np.random.choice(dicpeso[a], num_missing, replace=True)

In [66]:
prueba3

,patient_id,age,sex,weight,height
0,10874.0,60-70,0,65.0,NaN
1,20573.0,30-40,1,57.0,NaN
2,12198.0,50-60,0,81.0,NaN
3,20578.0,50-60,0,95.0,NaN
4,5104.0,60-70,1,59.0,160.0
...,...,...,...,...,...
15280,16237.0,40-50,0,71.0,NaN
15281,17361.0,40-50,0,91.0,NaN
15282,13456.0,>70,1,47.0,NaN
15283,16888.0,>70,1,72.0,NaN


In [67]:
prueba3['age'] = datat['age']

In [68]:
prueba3

,patient_id,age,sex,weight,height
0,10874.0,68.0,0,65.0,NaN
1,20573.0,34.0,1,57.0,NaN
2,12198.0,53.0,0,81.0,NaN
3,20578.0,59.0,0,95.0,NaN
4,5104.0,69.0,1,59.0,160.0
...,...,...,...,...,...
15280,16237.0,47.0,0,71.0,NaN
15281,17361.0,47.0,0,91.0,NaN
15282,13456.0,75.0,1,47.0,NaN
15283,16888.0,72.0,1,72.0,NaN


In [69]:
prueba3.isnull().sum()

patient_id        0
age               0
sex               0
weight            0
height        10436
dtype: int64

In [70]:
prueba3['height'] = np.where(prueba3['height'].isnull(), pipe.predict(prueba3.drop(columns=['patient_id','height'])), prueba3['height'])

In [71]:
prueba3

,patient_id,age,sex,weight,height
0,10874.0,68.0,0,65.0,170.645573
1,20573.0,34.0,1,57.0,160.106952
2,12198.0,53.0,0,81.0,174.608749
3,20578.0,59.0,0,95.0,177.705733
4,5104.0,69.0,1,59.0,160.000000
...,...,...,...,...,...
15280,16237.0,47.0,0,71.0,172.429854
15281,17361.0,47.0,0,91.0,177.020299
15282,13456.0,75.0,1,47.0,157.016823
15283,16888.0,72.0,1,72.0,162.813043


In [72]:
#Histograma peso

prueba3['weight'].iplot(kind='hist', bins=100, xTitle='Peso', yTitle='Frecuencia')

In [73]:
#histograma altura

prueba3['height'].iplot(kind='hist', bins=100, xTitle='Altura', yTitle='Frecuencia')

In [74]:
prueba3

,patient_id,age,sex,weight,height
0,10874.0,68.0,0,65.0,170.645573
1,20573.0,34.0,1,57.0,160.106952
2,12198.0,53.0,0,81.0,174.608749
3,20578.0,59.0,0,95.0,177.705733
4,5104.0,69.0,1,59.0,160.000000
...,...,...,...,...,...
15280,16237.0,47.0,0,71.0,172.429854
15281,17361.0,47.0,0,91.0,177.020299
15282,13456.0,75.0,1,47.0,157.016823
15283,16888.0,72.0,1,72.0,162.813043


In [76]:
datat[['weight', 'height']] = prueba3[['weight', 'height']]

In [77]:
datat

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,filename_hr,NOT_NORM,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int
0,16286,10874.0,68.0,0,170.645573,65.0,0.0,0.0,CS100 3,1996-07-04 14:12:49,records500/16000/16286_hr,1,76.564181,0.009693,0.191455,0.112,0.073455,0.146545
1,2647,20573.0,34.0,1,160.106952,57.0,0.0,0.0,CS-12 E,1989-04-09 08:52:33,records500/02000/02647_hr,0,48.891056,0.054899,0.440000,0.285,0.098857,0.134571
2,11732,12198.0,53.0,0,174.608749,81.0,0.0,0.0,CS-12 E,1994-01-26 13:21:53,records500/11000/11732_hr,0,60.514093,0.039332,0.416222,0.430,0.130889,0.070667
3,19751,20578.0,59.0,0,177.705733,95.0,0.0,0.0,CS100 3,1998-10-08 15:19:32,records500/19000/19751_hr,1,68.134493,0.006613,0.339200,0.155,0.096800,0.140800
4,7898,5104.0,69.0,1,160.000000,59.0,6.0,1.0,AT-6 C 5.8,1992-03-22 19:11:05,records500/07000/07898_hr,0,56.967957,0.012556,0.426250,0.048,0.357143,0.065750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,15377,16237.0,47.0,0,172.429854,71.0,0.0,0.0,CS-12 E,1995-11-25 10:18:13,records500/15000/15377_hr,0,59.087159,0.031838,0.402750,0.095,0.166000,0.079750
15281,5475,17361.0,47.0,0,177.020299,91.0,0.0,0.0,CS100 3,1990-12-06 09:29:17,records500/05000/05475_hr,0,76.600072,0.168882,0.349000,1.244,0.133500,0.130250
15282,3790,13456.0,75.0,1,157.016823,47.0,0.0,3.0,AT-6 C,1990-02-20 14:48:42,records500/03000/03790_hr,1,70.944370,0.015513,0.372545,0.346,0.115455,0.109091
15283,21643,16888.0,72.0,1,162.813043,72.0,0.0,0.0,CS100 3,2000-12-11 11:51:56,records500/21000/21643_hr,1,62.791343,0.040981,0.498889,0.279,0.163111,0.114444


In [79]:
datat = datat[['recording_date', 'ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site', 'device','BPM', 'HRV', 'QT_int', 'R_amplitude', 'QRS_width', 'PR_int']]

In [80]:
datat.isnull().sum()

recording_date    0
ecg_id            0
patient_id        0
age               0
sex               0
height            0
weight            0
nurse             0
site              0
device            0
BPM               0
HRV               0
QT_int            0
R_amplitude       0
QRS_width         0
PR_int            0
dtype: int64

In [81]:
datat

,recording_date,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int
0,1996-07-04 14:12:49,16286,10874.0,68.0,0,170.645573,65.0,0.0,0.0,CS100 3,76.564181,0.009693,0.191455,0.112,0.073455,0.146545
1,1989-04-09 08:52:33,2647,20573.0,34.0,1,160.106952,57.0,0.0,0.0,CS-12 E,48.891056,0.054899,0.440000,0.285,0.098857,0.134571
2,1994-01-26 13:21:53,11732,12198.0,53.0,0,174.608749,81.0,0.0,0.0,CS-12 E,60.514093,0.039332,0.416222,0.430,0.130889,0.070667
3,1998-10-08 15:19:32,19751,20578.0,59.0,0,177.705733,95.0,0.0,0.0,CS100 3,68.134493,0.006613,0.339200,0.155,0.096800,0.140800
4,1992-03-22 19:11:05,7898,5104.0,69.0,1,160.000000,59.0,6.0,1.0,AT-6 C 5.8,56.967957,0.012556,0.426250,0.048,0.357143,0.065750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,1995-11-25 10:18:13,15377,16237.0,47.0,0,172.429854,71.0,0.0,0.0,CS-12 E,59.087159,0.031838,0.402750,0.095,0.166000,0.079750
15281,1990-12-06 09:29:17,5475,17361.0,47.0,0,177.020299,91.0,0.0,0.0,CS100 3,76.600072,0.168882,0.349000,1.244,0.133500,0.130250
15282,1990-02-20 14:48:42,3790,13456.0,75.0,1,157.016823,47.0,0.0,3.0,AT-6 C,70.944370,0.015513,0.372545,0.346,0.115455,0.109091
15283,2000-12-11 11:51:56,21643,16888.0,72.0,1,162.813043,72.0,0.0,0.0,CS100 3,62.791343,0.040981,0.498889,0.279,0.163111,0.114444


In [83]:
datat = datat.drop(columns=['nurse', 'site'], axis = 1)

In [85]:
datat['NOT_NORM'] = dataset['NOT_NORM']

In [86]:
datat

,recording_date,ecg_id,patient_id,age,sex,height,weight,device,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int,NOT_NORM
0,1996-07-04 14:12:49,16286,10874.0,68.0,0,170.645573,65.0,CS100 3,76.564181,0.009693,0.191455,0.112,0.073455,0.146545,1
1,1989-04-09 08:52:33,2647,20573.0,34.0,1,160.106952,57.0,CS-12 E,48.891056,0.054899,0.440000,0.285,0.098857,0.134571,0
2,1994-01-26 13:21:53,11732,12198.0,53.0,0,174.608749,81.0,CS-12 E,60.514093,0.039332,0.416222,0.430,0.130889,0.070667,0
3,1998-10-08 15:19:32,19751,20578.0,59.0,0,177.705733,95.0,CS100 3,68.134493,0.006613,0.339200,0.155,0.096800,0.140800,1
4,1992-03-22 19:11:05,7898,5104.0,69.0,1,160.000000,59.0,AT-6 C 5.8,56.967957,0.012556,0.426250,0.048,0.357143,0.065750,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,1995-11-25 10:18:13,15377,16237.0,47.0,0,172.429854,71.0,CS-12 E,59.087159,0.031838,0.402750,0.095,0.166000,0.079750,0
15281,1990-12-06 09:29:17,5475,17361.0,47.0,0,177.020299,91.0,CS100 3,76.600072,0.168882,0.349000,1.244,0.133500,0.130250,0
15282,1990-02-20 14:48:42,3790,13456.0,75.0,1,157.016823,47.0,AT-6 C,70.944370,0.015513,0.372545,0.346,0.115455,0.109091,1
15283,2000-12-11 11:51:56,21643,16888.0,72.0,1,162.813043,72.0,CS100 3,62.791343,0.040981,0.498889,0.279,0.163111,0.114444,1


In [87]:
resultados

,record,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int
0,records500/16000/16286_hr,76.564181,0.009693,0.191455,0.112,0.073455,0.146545
0,records500/02000/02647_hr,48.891056,0.054899,0.440000,0.285,0.098857,0.134571
0,records500/11000/11732_hr,60.514093,0.039332,0.416222,0.430,0.130889,0.070667
0,records500/19000/19751_hr,68.134493,0.006613,0.339200,0.155,0.096800,0.140800
0,records500/07000/07898_hr,56.967957,0.012556,0.426250,0.048,0.357143,0.065750
...,...,...,...,...,...,...,...
0,records500/15000/15377_hr,59.087159,0.031838,0.402750,0.095,0.166000,0.079750
0,records500/05000/05475_hr,76.600072,0.168882,0.349000,1.244,0.133500,0.130250
0,records500/03000/03790_hr,70.944370,0.015513,0.372545,0.346,0.115455,0.109091
0,records500/21000/21643_hr,62.791343,0.040981,0.498889,0.279,0.163111,0.114444


In [88]:
datatest = pd.read_csv('./data/test_e1.csv')

In [89]:
datatest

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,filename_hr
0,14710,17114.0,58.0,0,NaN,NaN,0.0,0.0,CS100 3,1995-08-08 13:08:51,records500/14000/14710_hr
1,6122,321.0,76.0,1,NaN,NaN,11.0,1.0,AT-6 C 5.5,1991-05-02 15:04:23,records500/06000/06122_hr
2,20616,17562.0,70.0,1,NaN,NaN,1.0,2.0,CS-12,1999-09-22 21:03:57,records500/20000/20616_hr
3,7603,8062.0,62.0,0,NaN,NaN,0.0,0.0,CS100 3,1992-01-16 10:36:09,records500/07000/07603_hr
4,5768,9409.0,48.0,0,NaN,NaN,0.0,0.0,CS100 3,1991-02-01 13:32:35,records500/05000/05768_hr
...,...,...,...,...,...,...,...,...,...,...,...
6547,10113,6009.0,35.0,0,170.0,61.0,3.0,1.0,AT-6 C 5.5,1993-04-30 16:40:48,records500/10000/10113_hr
6548,873,5335.0,91.0,1,152.0,51.0,7.0,1.0,AT-6 C 5.5,1987-07-03 15:04:26,records500/00000/00873_hr
6549,11865,20063.0,77.0,0,NaN,NaN,NaN,3.0,AT-6 C,1994-03-03 08:22:09,records500/11000/11865_hr
6550,16731,17368.0,60.0,0,NaN,NaN,0.0,0.0,CS100 3,1996-08-29 06:48:15,records500/16000/16731_hr


In [91]:
resultadostest = pd.DataFrame()

for r in datatest['filename_hr']:
    resultadostest = pd.concat([resultadostest, ecg(r)], axis=0)

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:52: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/neurokit2/hrv/hrv_time.py:180: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareye

El archivo records500/12000/12080_hr contiene menos de 5 picos R, omitiendo.


/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfi

El archivo records500/10000/10266_hr contiene menos de 5 picos R, omitiendo.
El archivo records500/06000/06940_hr contiene menos de 5 picos R, omitiendo.


/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:37: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/var/folders/p_/7q2hf8lx21bd_px0wvspdh480000gn/T/ipykernel_12228/2746905579.py:47: RuntimeWarning:

Mean of empty slice.

/Users/josemariareyesfi

In [92]:
datatest = datatest.merge(resultadostest, left_on='filename_hr', right_on='record', how = 'left')

In [93]:
datatest

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,filename_hr,record,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int
0,14710,17114.0,58.0,0,NaN,NaN,0.0,0.0,CS100 3,1995-08-08 13:08:51,records500/14000/14710_hr,records500/14000/14710_hr,70.322769,0.006736,0.389600,0.071,0.202800,0.126400
1,6122,321.0,76.0,1,NaN,NaN,11.0,1.0,AT-6 C 5.5,1991-05-02 15:04:23,records500/06000/06122_hr,records500/06000/06122_hr,71.769711,0.136427,0.157091,0.771,0.090545,0.147091
2,20616,17562.0,70.0,1,NaN,NaN,1.0,2.0,CS-12,1999-09-22 21:03:57,records500/20000/20616_hr,records500/20000/20616_hr,113.326101,0.007985,0.284250,0.281,0.089375,0.128471
3,7603,8062.0,62.0,0,NaN,NaN,0.0,0.0,CS100 3,1992-01-16 10:36:09,records500/07000/07603_hr,records500/07000/07603_hr,78.700248,0.021756,0.413000,0.519,0.159167,0.108500
4,5768,9409.0,48.0,0,NaN,NaN,0.0,0.0,CS100 3,1991-02-01 13:32:35,records500/05000/05768_hr,records500/05000/05768_hr,79.139889,0.022187,0.209667,0.151,0.113000,0.106333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,10113,6009.0,35.0,0,170.0,61.0,3.0,1.0,AT-6 C 5.5,1993-04-30 16:40:48,records500/10000/10113_hr,records500/10000/10113_hr,80.601158,0.008690,0.344000,0.226,0.084333,0.102500
6548,873,5335.0,91.0,1,152.0,51.0,7.0,1.0,AT-6 C 5.5,1987-07-03 15:04:26,records500/00000/00873_hr,records500/00000/00873_hr,71.167983,0.117797,0.192364,0.543,0.133273,0.156909
6549,11865,20063.0,77.0,0,NaN,NaN,NaN,3.0,AT-6 C,1994-03-03 08:22:09,records500/11000/11865_hr,records500/11000/11865_hr,70.400055,0.005921,0.431636,0.855,0.055455,0.173636
6550,16731,17368.0,60.0,0,NaN,NaN,0.0,0.0,CS100 3,1996-08-29 06:48:15,records500/16000/16731_hr,records500/16000/16731_hr,66.861877,0.222505,0.323800,0.374,0.094600,0.169200


In [95]:
datat = datat.drop(columns=['record'])

,recording_date,ecg_id,patient_id,age,sex,height,weight,device,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int,NOT_NORM
0,1996-07-04 14:12:49,16286,10874.0,68.0,0,170.645573,65.0,CS100 3,76.564181,0.009693,0.191455,0.112,0.073455,0.146545,1
1,1989-04-09 08:52:33,2647,20573.0,34.0,1,160.106952,57.0,CS-12 E,48.891056,0.054899,0.440000,0.285,0.098857,0.134571,0
2,1994-01-26 13:21:53,11732,12198.0,53.0,0,174.608749,81.0,CS-12 E,60.514093,0.039332,0.416222,0.430,0.130889,0.070667,0
3,1998-10-08 15:19:32,19751,20578.0,59.0,0,177.705733,95.0,CS100 3,68.134493,0.006613,0.339200,0.155,0.096800,0.140800,1
4,1992-03-22 19:11:05,7898,5104.0,69.0,1,160.000000,59.0,AT-6 C 5.8,56.967957,0.012556,0.426250,0.048,0.357143,0.065750,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,1995-11-25 10:18:13,15377,16237.0,47.0,0,172.429854,71.0,CS-12 E,59.087159,0.031838,0.402750,0.095,0.166000,0.079750,0
15281,1990-12-06 09:29:17,5475,17361.0,47.0,0,177.020299,91.0,CS100 3,76.600072,0.168882,0.349000,1.244,0.133500,0.130250,0
15282,1990-02-20 14:48:42,3790,13456.0,75.0,1,157.016823,47.0,AT-6 C,70.944370,0.015513,0.372545,0.346,0.115455,0.109091,1
15283,2000-12-11 11:51:56,21643,16888.0,72.0,1,162.813043,72.0,CS100 3,62.791343,0.040981,0.498889,0.279,0.163111,0.114444,1


In [96]:
# Clasificadores

X = datat.drop(columns=['NOT_NORM', 'recording_date', 'ecg_id', 'patient_id', 'device'])

y = datat['NOT_NORM']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

models = {'LR': LogisticRegression(), 'SGDClassifier': SGDClassifier(), 'LDA': LinearDiscriminantAnalysis(), 'SVC': SVC(), 'KNN': KNeighborsClassifier(), 'DTC': DecisionTreeClassifier(), 'RFC': RandomForestClassifier(), 'XGB': XGBClassifier(), 'MLP': MLPClassifier()}
scalers = {'StandardScaler': StandardScaler(), 'MinMaxScaler': MinMaxScaler(), 'RobustScaler': RobustScaler()}

results = pd.DataFrame()

for model_name, model in models.items():
    for scaler_name, scaler in scalers.items():
        pipe = Pipeline(steps=[('scaler', scaler), ('model', model)])
        pipe.fit(X_train, y_train)
        #Cross validation
        cv_score = cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')
        scoretrain = pipe.score(X_train, y_train)   
        scoretest = pipe.score(X_test, y_test)
        new_row = pd.DataFrame({'model': [model_name], 'scaler': [scaler_name], 'train_score': [scoretrain], 'test_score': [scoretest], 'cv_score': [cv_score.mean()], 'cv_score_std': [cv_score.std()]})
        results = pd.concat([results, new_row], ignore_index=True)

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/j

In [97]:
results.sort_values(by='test_score', ascending=False)

,model,scaler,train_score,test_score,cv_score,cv_score_std
20,RFC,RobustScaler,1.000000,0.746991,0.743784,0.005837
25,MLP,MinMaxScaler,0.748844,0.746468,0.742999,0.006434
9,SVC,StandardScaler,0.760010,0.746468,0.744046,0.005561
24,MLP,StandardScaler,0.768734,0.746206,0.744395,0.004569
11,SVC,RobustScaler,0.756783,0.745160,0.741778,0.003579
26,MLP,RobustScaler,0.773881,0.744375,0.744569,0.005648
19,RFC,MinMaxScaler,1.000000,0.744113,0.738811,0.004953
18,RFC,StandardScaler,1.000000,0.738880,0.739771,0.005563
10,SVC,MinMaxScaler,0.730960,0.737310,0.729653,0.008188
23,XGB,RobustScaler,0.924889,0.736264,0.732269,0.005016


In [101]:
datatest.isnull().sum()

ecg_id               0
patient_id           0
age                 28
sex                  0
height            4418
weight            3710
nurse              479
site                 3
device               0
recording_date       0
filename_hr          0
record               0
BPM                  3
HRV                  3
QT_int              24
R_amplitude          3
QRS_width           27
PR_int               7
dtype: int64

In [102]:
datatest['age'] = datatest['age'].fillna(datatest['age'].mode()[0])

In [103]:
missings = ['BPM', 'HRV', 'QT_int', 'R_amplitude', 'QRS_width', 'PR_int']

datatest[missings] = datatest[missings].fillna(datatest[missings].median())

In [104]:
datatest['nurse'] = datatest['nurse'].fillna(datatest['nurse'].mode()[0])

datatest['site'] = datatest['site'].fillna(datatest['site'].mode()[0])

In [105]:
datatest.isnull().sum()

ecg_id               0
patient_id           0
age                  0
sex                  0
height            4418
weight            3710
nurse                0
site                 0
device               0
recording_date       0
filename_hr          0
record               0
BPM                  0
HRV                  0
QT_int               0
R_amplitude          0
QRS_width            0
PR_int               0
dtype: int64

In [106]:
prueba3 = datatest[['patient_id', 'age', 'sex', 'weight', 'height']].copy()

prueba3

,patient_id,age,sex,weight,height
0,17114.0,58.0,0,NaN,NaN
1,321.0,76.0,1,NaN,NaN
2,17562.0,70.0,1,NaN,NaN
3,8062.0,62.0,0,NaN,NaN
4,9409.0,48.0,0,NaN,NaN
...,...,...,...,...,...
6547,6009.0,35.0,0,61.0,170.0
6548,5335.0,91.0,1,51.0,152.0
6549,20063.0,77.0,0,NaN,NaN
6550,17368.0,60.0,0,NaN,NaN


In [107]:
for a in prueba3['age']:
    if a < 18:
        prueba3['age'] = prueba3['age'].replace(a, '<18')
    elif a >= 18 and a < 30:
        prueba3['age'] = prueba3['age'].replace(a, '18-30')
    elif a >= 30 and a < 40:
        prueba3['age'] = prueba3['age'].replace(a, '30-40')
    elif a >= 40 and a < 50:
        prueba3['age'] = prueba3['age'].replace(a, '40-50')
    elif a >= 50 and a < 60:
        prueba3['age'] = prueba3['age'].replace(a, '50-60')
    elif a >= 60 and a < 70:
        prueba3['age'] = prueba3['age'].replace(a, '60-70')
    else:
        prueba3['age'] = prueba3['age'].replace(a, '>70')

In [108]:
prueba3

,patient_id,age,sex,weight,height
0,17114.0,50-60,0,NaN,NaN
1,321.0,>70,1,NaN,NaN
2,17562.0,>70,1,NaN,NaN
3,8062.0,60-70,0,NaN,NaN
4,9409.0,40-50,0,NaN,NaN
...,...,...,...,...,...
6547,6009.0,30-40,0,61.0,170.0
6548,5335.0,>70,1,51.0,152.0
6549,20063.0,>70,0,NaN,NaN
6550,17368.0,60-70,0,NaN,NaN


In [109]:
dicpeso = {'<18': prueba3[prueba3['age'] == '<18']['weight'].dropna(), '18-30': prueba3[prueba3['age'] == '18-30']['weight'].dropna(), '30-40': prueba3[prueba3['age'] == '30-40']['weight'].dropna(), '40-50': prueba3[prueba3['age'] == '40-50']['weight'].dropna(), '50-60': prueba3[prueba3['age'] == '50-60']['weight'].dropna(), '60-70': prueba3[prueba3['age'] == '60-70']['weight'].dropna(), '>70': prueba3[prueba3['age'] == '>70']['weight'].dropna()}

In [110]:
#Donde haya nulos en weight, imputa una muestra aleatoria del grupo edad correspondiente en dicpeso

for a in dicpeso.keys():
    # Encuentra el número de valores nulos para la edad actual
    num_missing = prueba3[(prueba3['age'] == a) & (prueba3['weight'].isnull())].shape[0]
    
    if num_missing > 0:  # Solo proceder si hay valores nulos
        # Imputar valores aleatorios desde dicpeso[a]
        prueba3.loc[(prueba3['age'] == a) & (prueba3['weight'].isnull()), 'weight'] = np.random.choice(dicpeso[a], num_missing, replace=True)

In [111]:
prueba3

,patient_id,age,sex,weight,height
0,17114.0,50-60,0,70.0,NaN
1,321.0,>70,1,74.0,NaN
2,17562.0,>70,1,89.0,NaN
3,8062.0,60-70,0,63.0,NaN
4,9409.0,40-50,0,72.0,NaN
...,...,...,...,...,...
6547,6009.0,30-40,0,61.0,170.0
6548,5335.0,>70,1,51.0,152.0
6549,20063.0,>70,0,60.0,NaN
6550,17368.0,60-70,0,114.0,NaN


In [112]:
prueba3.isnull().sum()

patient_id       0
age              0
sex              0
weight           0
height        4418
dtype: int64

In [123]:
prueba3['age'] = datatest['age']

In [124]:
prueba3

,patient_id,age,sex,weight,height
0,17114.0,58.0,0,70.0,NaN
1,321.0,76.0,1,74.0,NaN
2,17562.0,70.0,1,89.0,NaN
3,8062.0,62.0,0,63.0,NaN
4,9409.0,48.0,0,72.0,NaN
...,...,...,...,...,...
6547,6009.0,35.0,0,61.0,170.0
6548,5335.0,91.0,1,51.0,152.0
6549,20063.0,77.0,0,60.0,NaN
6550,17368.0,60.0,0,114.0,NaN


In [125]:
prueba3['height'] = np.where(prueba3['height'].isnull(), pipe.predict(prueba3.drop(columns=['patient_id','height'])), prueba3['height'])

In [126]:
prueba3

,patient_id,age,sex,weight,height
0,17114.0,58.0,0,70.0,171.968146
1,321.0,76.0,1,74.0,163.177327
2,17562.0,70.0,1,89.0,166.689311
3,8062.0,62.0,0,63.0,170.304961
4,9409.0,48.0,0,72.0,172.622394
...,...,...,...,...,...
6547,6009.0,35.0,0,61.0,170.000000
6548,5335.0,91.0,1,51.0,152.000000
6549,20063.0,77.0,0,60.0,169.323590
6550,17368.0,60.0,0,114.0,181.873743


In [127]:
datatest[['weight', 'height']] = prueba3[['weight', 'height']]

In [129]:
datatest.isnull().sum()

ecg_id            0
patient_id        0
age               0
sex               0
height            0
weight            0
nurse             0
site              0
device            0
recording_date    0
filename_hr       0
record            0
BPM               0
HRV               0
QT_int            0
R_amplitude       0
QRS_width         0
PR_int            0
dtype: int64

In [130]:
datatest = datatest.drop(columns=['nurse', 'site', 'filename_hr'], axis = 1)

In [133]:
datatest = datatest[['recording_date', 'ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'device','BPM', 'HRV', 'QT_int', 'R_amplitude', 'QRS_width', 'PR_int']]

In [132]:
datat

,recording_date,ecg_id,patient_id,age,sex,height,weight,device,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int,NOT_NORM
0,1996-07-04 14:12:49,16286,10874.0,68.0,0,170.645573,65.0,CS100 3,76.564181,0.009693,0.191455,0.112,0.073455,0.146545,1
1,1989-04-09 08:52:33,2647,20573.0,34.0,1,160.106952,57.0,CS-12 E,48.891056,0.054899,0.440000,0.285,0.098857,0.134571,0
2,1994-01-26 13:21:53,11732,12198.0,53.0,0,174.608749,81.0,CS-12 E,60.514093,0.039332,0.416222,0.430,0.130889,0.070667,0
3,1998-10-08 15:19:32,19751,20578.0,59.0,0,177.705733,95.0,CS100 3,68.134493,0.006613,0.339200,0.155,0.096800,0.140800,1
4,1992-03-22 19:11:05,7898,5104.0,69.0,1,160.000000,59.0,AT-6 C 5.8,56.967957,0.012556,0.426250,0.048,0.357143,0.065750,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,1995-11-25 10:18:13,15377,16237.0,47.0,0,172.429854,71.0,CS-12 E,59.087159,0.031838,0.402750,0.095,0.166000,0.079750,0
15281,1990-12-06 09:29:17,5475,17361.0,47.0,0,177.020299,91.0,CS100 3,76.600072,0.168882,0.349000,1.244,0.133500,0.130250,0
15282,1990-02-20 14:48:42,3790,13456.0,75.0,1,157.016823,47.0,AT-6 C,70.944370,0.015513,0.372545,0.346,0.115455,0.109091,1
15283,2000-12-11 11:51:56,21643,16888.0,72.0,1,162.813043,72.0,CS100 3,62.791343,0.040981,0.498889,0.279,0.163111,0.114444,1


In [134]:
#Mejor clasificador: RandomForestClassifier RobustScaler

X = datat.drop(columns=['NOT_NORM', 'recording_date', 'ecg_id', 'patient_id', 'device'])

y = datat['NOT_NORM']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

pipe = Pipeline(steps=[('scaler', RobustScaler()), ('model', RandomForestClassifier())])

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('model', RandomForestClassifier())])

In [135]:
#score en train

pipe.score(X_train, y_train)

0.9999127628020588

In [136]:
#score en test

pipe.score(X_test, y_test)

0.7422815279958137

In [138]:
#Predicción

datatest['NOT_NORM'] = pipe.predict(datatest.drop(columns=['recording_date', 'ecg_id', 'patient_id', 'device']))

In [140]:
datatest

,recording_date,ecg_id,patient_id,age,sex,height,weight,device,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int,NOT_NORM
0,1995-08-08 13:08:51,14710,17114.0,58.0,0,171.968146,70.0,CS100 3,70.322769,0.006736,0.389600,0.071,0.202800,0.126400,0
1,1991-05-02 15:04:23,6122,321.0,76.0,1,163.177327,74.0,AT-6 C 5.5,71.769711,0.136427,0.157091,0.771,0.090545,0.147091,1
2,1999-09-22 21:03:57,20616,17562.0,70.0,1,166.689311,89.0,CS-12,113.326101,0.007985,0.284250,0.281,0.089375,0.128471,1
3,1992-01-16 10:36:09,7603,8062.0,62.0,0,170.304961,63.0,CS100 3,78.700248,0.021756,0.413000,0.519,0.159167,0.108500,1
4,1991-02-01 13:32:35,5768,9409.0,48.0,0,172.622394,72.0,CS100 3,79.139889,0.022187,0.209667,0.151,0.113000,0.106333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,1993-04-30 16:40:48,10113,6009.0,35.0,0,170.000000,61.0,AT-6 C 5.5,80.601158,0.008690,0.344000,0.226,0.084333,0.102500,0
6548,1987-07-03 15:04:26,873,5335.0,91.0,1,152.000000,51.0,AT-6 C 5.5,71.167983,0.117797,0.192364,0.543,0.133273,0.156909,1
6549,1994-03-03 08:22:09,11865,20063.0,77.0,0,169.323590,60.0,AT-6 C,70.400055,0.005921,0.431636,0.855,0.055455,0.173636,1
6550,1996-08-29 06:48:15,16731,17368.0,60.0,0,181.873743,114.0,CS100 3,66.861877,0.222505,0.323800,0.374,0.094600,0.169200,1


In [142]:
#Predict proba

datatest['NOT_NORM_PROBA'] = pipe.predict_proba(datatest.drop(columns=['NOT_NORM','recording_date', 'ecg_id', 'patient_id', 'device']))[:,1]

In [143]:
datatest

,recording_date,ecg_id,patient_id,age,sex,height,weight,device,BPM,HRV,QT_int,R_amplitude,QRS_width,PR_int,NOT_NORM,NOT_NORM_PROBA
0,1995-08-08 13:08:51,14710,17114.0,58.0,0,171.968146,70.0,CS100 3,70.322769,0.006736,0.389600,0.071,0.202800,0.126400,0,0.50
1,1991-05-02 15:04:23,6122,321.0,76.0,1,163.177327,74.0,AT-6 C 5.5,71.769711,0.136427,0.157091,0.771,0.090545,0.147091,1,0.86
2,1999-09-22 21:03:57,20616,17562.0,70.0,1,166.689311,89.0,CS-12,113.326101,0.007985,0.284250,0.281,0.089375,0.128471,1,0.61
3,1992-01-16 10:36:09,7603,8062.0,62.0,0,170.304961,63.0,CS100 3,78.700248,0.021756,0.413000,0.519,0.159167,0.108500,1,0.80
4,1991-02-01 13:32:35,5768,9409.0,48.0,0,172.622394,72.0,CS100 3,79.139889,0.022187,0.209667,0.151,0.113000,0.106333,1,0.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,1993-04-30 16:40:48,10113,6009.0,35.0,0,170.000000,61.0,AT-6 C 5.5,80.601158,0.008690,0.344000,0.226,0.084333,0.102500,0,0.25
6548,1987-07-03 15:04:26,873,5335.0,91.0,1,152.000000,51.0,AT-6 C 5.5,71.167983,0.117797,0.192364,0.543,0.133273,0.156909,1,0.82
6549,1994-03-03 08:22:09,11865,20063.0,77.0,0,169.323590,60.0,AT-6 C,70.400055,0.005921,0.431636,0.855,0.055455,0.173636,1,0.91
6550,1996-08-29 06:48:15,16731,17368.0,60.0,0,181.873743,114.0,CS100 3,66.861877,0.222505,0.323800,0.374,0.094600,0.169200,1,0.74


In [144]:
e2 = datatest[['ecg_id', 'NOT_NORM_PROBA']].copy()

e2.columns = ['ID', 'y_hat']

In [145]:
e2

,ID,y_hat
0,14710,0.50
1,6122,0.86
2,20616,0.61
3,7603,0.80
4,5768,0.91
...,...,...
6547,10113,0.25
6548,873,0.82
6549,11865,0.91
6550,16731,0.74


In [146]:
e2.to_csv('./data/e2.csv', index=False)